# Google API key initialization

In [1]:
import os
with open(r'C:\Python\HQgeminiAPIKey.txt', 'r')as file:
    GOOGLE_API_KEY = file.read()
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
with open(r'C:\Python\geminiAPIKey.txt', 'r')as file:
    GOOGL_EMBED_API_KEY = file.read()

# Multimodel RAG Chatbot

In [2]:
from glob import glob
from datetime import datetime as dt
import time
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings as gemini_embed
gemini = ChatGoogleGenerativeAI(model = 'gemini-2.5-flash')
print("Gemini AI Model initiated!")
embedding = gemini_embed(model='models/gemini-embedding-001', google_api_key=GOOGL_EMBED_API_KEY )
print("Gemini Embedding Model initiated!")
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader, UnstructuredPDFLoader
from langchain_community.document_loaders.parsers import TesseractBlobParser
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough
from IPython.display import display, Markdown

G:\Python\genai venv\genai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Gemini AI Model initiated!
Gemini Embedding Model initiated!


# Loading and processing PDF HR policy

In [3]:
# Function to read PDF file, summarise and chunk
def chunking(file_path, chunks = 1000, overlap = 200):
    print("Reading policies of {0}".format(file_path.split('\\')[-1]))                
    loader = UnstructuredPDFLoader(file_path = file_path, stratergy='hi-res', infer_table_structure=True, extract_images=True,
                                   image_output_dir_path='./images', mode='paged', chunking_strategy="by_title")
    doc = loader.load()
    docs = [d.page_content for d in doc]
    print("Policy reading of {0} has completed!".format(file_path.split('\\')[-1]) )
    print()
    print('Getting the summary of leave policy {0}'.format(file_path.split('\\')[-1]))
    summary = gemini.invoke([
    ('system',
     '''You are a best document summarizer of HR department for a company who can summarize brief and short by highlighting only key points. 
     You will get the doc which is a HR policy from which you have to briefly summarize the policy'''),
    ("human",
     docs)]).content
    metadata = {'source':'Gemini AI',
                'filename' : file_path.split('\\')[-1],
                'category' : 'Summary'}
    summary_doc = Document(metadata=metadata, page_content=summary)
    doc.append(summary_doc)
    print("Added the summary of {0} which is as below\n********************************************\n\n".format(file_path.split('\\')[-1]))
    display(Markdown(summary))
    print('\n\n********************************************')      
    print()
    print('Getting the definition of important terminologies from {0}'.format(file_path.split('\\')[-1]))
    definitions = gemini.invoke([
    ('system',
     '''You are a senior HR key point identifier from given documents and you are good at finding important terminologies from the given text document and
     if these terminologies have no any explanation or definition available in the given text then you get the brief and precise definition. Please refer the given example.
     Example 1, Privilege leave (PL), also known as earned leave, is a paid type of leave that employees accrue based on their days worked.
     Example 2. FT Employees, refers to full-time employees, who are typically permanent staff working 30–40+ hours per week and are eligible for a full benefits package, including health insurance and paid time off.'''),
    ("human",
     docs)]).content
    metadata = {'source':'Gemini AI',
                'filename' : file_path.split('\\')[-1],
                'category' : 'Definitions'}
    definitions_doc = Document(metadata=metadata, page_content=definitions)
    doc.append(definitions_doc)
    print("Added the definitions of key terminologies from {0} which is as below\n********************************************\n\n{1}\n\n********************************************".
        format(file_path.split('\\')[-1], display(Markdown(definitions))))
    #print()
    print()
    print("Chunking the policy of {0}".format(file_path.split('\\')[-1]))
    print()
    print('Filtering complex metadata from the document!')
    # Filtering extra metadata from chunk files
    filtered_chunks = filter_complex_metadata(doc)
    print("Complex metadata has been Filtered!")
    print()
    print('Chunking the document with chunk size {0} words and overlapping text with {1} words!'.format(chunks,overlap))
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunks, chunk_overlap=overlap)
    doc_chunks = splitter.split_documents(filtered_chunks)
    print("HR Policy processing of {0} has completed!".format(file_path.split('\\')[-1]))

    return doc_chunks


# Initializing ChromaDB : **Please run this only once**
def init_ChrmDB(filtered_chunks):
    if os.path.isdir('ChromaDB'):
        chromadb = Chroma(persist_directory="./ChromaDB",
                          embedding_function=embedding,
                          collection_name="HQHR")
        if len(filtered_chunks)==1:
            chromadb.add_documents(documents=filtered_chunks[0])
        else:
            for chunk in filtered_chunks:
                time.sleep(10)
                print("{0} : Waiting for 10 seconds for embedding {1} policy".
                    format(dt.now().strftime("%d-%m-%Y %H:%M:%S"),chunk[0].metadata['filename']))
                chromadb.add_documents(documents=chunk)
    else:
        if len(filtered_chunks)==1:
            chromadb = Chroma.from_documents(documents=filtered_chunks[0],
                                             persist_directory="./ChromaDB",
                                             collection_name='HQHR',
                                             collection_metadata = {"hnsw:space":"cosine"},
                                             embedding=embedding)
        else:
            for chunk in filtered_chunks:
                time.sleep(10)
                print("{0} : Waiting for 10 seconds for embedding {1} policy".
                    format(dt.now().strftime("%d-%m-%Y %H:%M:%S"),chunk[0].metadata['filename']))
                if os.path.isdir('ChromaDB'):
                    chromadb.add_documents(documents=chunk)
                else:
                    chromadb = Chroma.from_documents(documents=chunk,
                                                     persist_directory="./ChromaDB",
                                                     collection_name='HQHR',
                                                     collection_metadata = {"hnsw:space":"cosine"},
                                                     embedding=embedding)

    chromadb = Chroma(persist_directory="./ChromaDB",
                          embedding_function=embedding,
                          collection_name="HQHR")
    print("Database created and embedded data added to Chroma DB with total policy collection of {0}.".format(chromadb._collection.count()))

    return chromadb

# Chunkin and Creating ChromaDB with Embeddings
- Please run below cell only once

In [4]:
# Please run this cell only once.

chunk_files = []
paths = glob(r'C:\Users\akash\Documents\Haqdarshak\Work\hqhr_chatbot\HR_Policies\*.pdf')
for path in paths:
    chunk_files.append(chunking(file_path=path))

chroma_db = init_ChrmDB(chunk_files)

Reading policies of Haqdarshak Maternity Benefits Policy - final revised V10.pdf


The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.unstructured.io/open-source/core-functionality/chunking


Policy reading of Haqdarshak Maternity Benefits Policy - final revised V10.pdf has completed!

Getting the summary of leave policy Haqdarshak Maternity Benefits Policy - final revised V10.pdf
Added the summary of Haqdarshak Maternity Benefits Policy - final revised V10.pdf which is as below
********************************************




Here's a brief summary of HESPL's Maternity Benefits Policy, highlighting key points:

**HESPL - MATERNITY BENEFITS POLICY (Effective August 1, 2020)**

This policy provides maternity benefits to women employees in line with relevant acts.

**Key Points:**

*   **Eligibility:** Full-time women employees who have worked for at least 80 days in the 12 months preceding their expected delivery date. (Excludes ESIC-covered employees for monetary benefits).
*   **Leave Duration:**
    *   **26 weeks** for two surviving children (up to 8 weeks before expected delivery).
    *   **12 weeks** for more than two children.
    *   **12 weeks** for adopting a child below three months or for commissioning mothers.
*   **Salary & Benefits:**
    *   Full salary and benefits continue during maternity leave.
    *   No impact on appraisal or review eligibility.
    *   Post-leave, Work From Home (WFH) or part-time options may be considered with adjusted compensation, based on mutual agreement and management approval.
*   **Medical Bonus:** A one-time medical bonus of **Rs. 3500** is provided if the employer does not offer pre-natal, confinement, and post-natal care.
*   **Special Cases:**
    *   **Miscarriage/Medical Termination:** 6 weeks leave with wages.
    *   **Tubectomy Operation:** 2 weeks leave with wages.
    *   **Pregnancy-related Illness:** Up to 1 month leave with wages.
*   **Resuming Work:**
    *   Employees are expected to resume full-time work after 6 months.
    *   Failure to resume may lead to a review of engagement terms or discontinuation (last month of ML treated as notice).
    *   Extended leave requests require prior intimation and are subject to management discretion.
*   **Work Restrictions:** Pregnant employees will not be assigned arduous tasks, long hours of standing, or work that may interfere with pregnancy, especially 1 month before the 6-week pre-delivery period.
*   **Dismissal of Benefits:** Benefits can be dismissed for misconduct, breach of agreements (e.g., Non-Compete), or working for another establishment during maternity leave.
*   **Documentation:** Medical certificates confirming pregnancy, expected delivery date, or any contingency (miscarriage, illness) are required for leave applications.



********************************************

Getting the definition of important terminologies from Haqdarshak Maternity Benefits Policy - final revised V10.pdf


...

Here's a brief summary of the Work From Office (WFO) SOP:

**Work From Office (WFO) - Standard Operating Procedure (SOP)**

*   **Effective Date:** 1st June 2023.
*   **Scope:** Applies to Pune, Delhi, and Bangalore administrative offices.
*   **Mandatory WFO Days:**
    *   **Monday, Tuesday, Wednesday:** Compulsory WFO for all employees associated with these offices.
*   **Optional WFO/WFH Days:**
    *   **Thursday & Friday:** Employees can choose to Work From Office or Work From Home.
*   **Attendance Tracking:**
    *   **Office Register:** Sign daily when using office space (tracks office usage).
    *   **Zoho Clock-in/out:** **Mandatory daily** for salary processing and official attendance records.
*   **Exceptions:**
    *   For those needing more time to start WFO (e.g., returning to base location).
    *   Requires approval from your **Department Head** (email Vertical Head, marking LM and HR).
    *   **HR must be notified in advance.**
*   **Remote Employees (not mapped to these 3 offices):** Continue WFH until co-working space access is provided.
*   **Queries:** Contact the HR department.



********************************************

Getting the definition of important terminologies from work from office - SOP.pdf


Here are the HR key points and terminologies identified from the provided document, along with their explanations or precise definitions where not explicitly detailed in the text:

1.  **Work From Office (WFO) - SOP:** A Standard Operating Procedure (SOP) outlining the protocols and procedures employees must follow for resuming work from the administrative offices.
2.  **Standard Operating Procedure (SOP):** A set of step-by-step instructions compiled by an organization to help workers carry out routine operations, ensuring consistency and quality.
3.  **Administrative Offices:** Refers to the primary physical office locations (Pune, Delhi, and Bangalore) where the Work From Office SOP is applicable.
4.  **Compulsory Work from Office Days:** The designated days (Monday, Tuesday, and Wednesday) when all employees associated with the administrative offices are required to work from the office.
5.  **Optional Work from Office Days:** The designated days (Thursday and Friday) when employees have the flexibility to choose between working from the office or working from home.
6.  **Signing the Register:** A daily requirement for employees to physically sign a register when using office space, used to track office presence.
7.  **Zoho Log in & Log Out:** The process of clocking in and out digitally on the Zoho platform, used to record attendance and leave data for salary processing purposes.
8.  **Salary Processing:** The administrative function of calculating and distributing employee wages, including deductions and benefits, based on attendance and other relevant data.
9.  **Register Attendance:** The data collected from the physical sign-in register, used specifically to track office space usage and monitor compliance with the Work From Office SOP.
10. **Exceptions:** Special approvals granted to employees who require additional time to start working from the office due to specific circumstances, requiring approval from the Department Head and notification to HR.
11. **Department Head:** A senior employee responsible for overseeing a specific functional area or team within an organization, typically with authority to make decisions and approve requests related to their department.
12. **HR (Human Resources):** The department within an organization responsible for managing employee-related functions, including policy enforcement, employee relations, and handling queries.
13. **Line Manager (LM):** An employee who directly supervises and manages a team or individual contributors, responsible for their daily tasks and performance, and who should be included in exception request communications.
14. **Vertical Head:** A senior leader responsible for an entire business unit or a major functional division (vertical) within an organization, typically overseeing multiple teams or departments, and who approves exception requests.
15. **Base Location:** The primary physical office or administrative location to which an employee is officially assigned or typically reports for work.
16. **Co-work Space Access:** The ability for employees to utilize shared office environments or flexible workspaces, typically provided by a third party, often on a membership or pay-per-use basis, intended for employees not mapped to a specific company office.
17. **Work From Home (WFH):** A flexible work arrangement where employees perform their job duties from their residence or another remote location outside the traditional office environment.

Added the definitions of key terminologies from work from office - SOP.pdf which is as below
********************************************

None

********************************************

Chunking the policy of work from office - SOP.pdf

Filtering complex metadata from the document!
Complex metadata has been Filtered!

Chunking the document with chunk size 1000 words and overlapping text with 200 words!
HR Policy processing of work from office - SOP.pdf has completed!


KeyError: 'element_id'

In [4]:
# Loading Embeddings from database
chroma_db = Chroma(collection_name="HQHR",
                   embedding_function=embedding,
                   persist_directory="./ChromaDB"
                  )

In [20]:
prompt='''
You are Harshil, an HR assistant for Haqdarshak company. Your sole purpose is to answer employee queries using **only** the provided context.

**Strict Instructions for Response Generation:**

1.  **Context Reliance:** Base your answer **strictly and exclusively** on the provided `Context`. Do not use external knowledge or invent facts.
2.  **Answering Style:**
    * Be **direct, simple, and concise**. The answer should ideally be $\le 2$ lines.
    * **NEVER** generate introductory or concluding sentences. Start directly with the answer.
    * **Example:** Q: How many PL a year? A: 16 Days
3.  **Handling Missing Information:**
    * If the answer is **not found** in the `Context`, reply with: "I cannot answer this from the current knowledge base."
    * **Do not** provide external or general knowledge (e.g., the definition of Casual Leave). This violates Instruction 2 and consumes unnecessary tokens.
4.  **Token Efficiency:** Prioritize short, accurate responses to minimize token usage.

Question: {question}

Context: {context}

Answer:
'''

def format_res(res):
    return "\n\n".join(content.page_content for content in res)


# Symantic Retrieval, chat prompt template and chat RAG chain
def retriever_chat_rag_chain(k=5, query="Haqarshak Empowerment Solutions Pvt. Ltd."):
    # Symantic Retrieval
    symantic_retriever = chroma_db.as_retriever(search_type='similarity', search_kwargs={'k':k})

    # Chat prompt template
    rag_prompt_template = ChatPromptTemplate.from_template(prompt)

    # Chat RAG chain
    chat_rag_chain=({'context':(symantic_retriever | format_res),
                     'question':RunnablePassthrough()} | rag_prompt_template | gemini)

    # Storing the reponse
    response = chat_rag_chain.invoke(query)
    
    return response, k

In [19]:
retriever_chat_rag_chain(query="Leave Types")

(AIMessage(content='Casual Leave (CL), Privilege Leave (PL), Medical Leave, Leave Without Pay (LWP), Sabbatical, Compensatory Off (Comp Off).', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--35bf648d-bc96-439b-a7a4-6871a56582b4-0', usage_metadata={'input_tokens': 1157, 'output_tokens': 222, 'total_tokens': 1379, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 191}}),
 5)

In [24]:
html_code = """
<p style="color: hotpink; background-color: purple; padding: 1px; display: inline-block; font-size: 10px; font-weight: bold;">
    हक़
</p>
"""
'''chat_response = chat_rag_chain.invoke("Introduce yourself in professional manner and ask politely how you can help")
display(Markdown(html_code+" : "+chat_response.content))
print('**********************************************\n')'''

INITIAL_GREETING = "Hello! I'm Harshil, your HR assistant for Haqdarshak company. I can answer your queries about HR policies. What can I help you with today?\n\n"
def chtbot():
    display(Markdown(html_code+" : "+INITIAL_GREETING))
    
    while True:

        usrQuery = input(" Please ask your query related to Haqdarshak's HR Policies. \n If you are done then type Exit or Quit to finish.\n\n🗣️ :")
        if usrQuery.lower() in ['exit', 'quit']:
            break
        else:
            print('\n')
            #chat_response = chat_rag_chain.invoke(query)
            chat_response, ks = retriever_chat_rag_chain(query=usrQuery)
            
            while chat_response == 'I cannot answer this from the current knowledge base.':
                if ks>20:
                    break
                else:
                    ks+=3
                    chat_response, ks = retriever_chat_rag_chain(k=ks, query=usrQuery)
                    #chat_response = chat_rag_chain.invoke(query)
                    
            display(Markdown(html_code+" : "+chat_response.content))
            print('**********************************************\n')

chtbot()


<p style="color: hotpink; background-color: purple; padding: 1px; display: inline-block; font-size: 10px; font-weight: bold;">
    हक़
</p>
 : Hello! I'm Harshil, your HR assistant for Haqdarshak company. I can answer your queries about HR policies. What can I help you with today?



 Please ask your query related to Haqdarshak's HR Policies. 
 If you are done then type Exit or Quit to finish.

🗣️ : Leave types



<p style="color: hotpink; background-color: purple; padding: 1px; display: inline-block; font-size: 10px; font-weight: bold;">
    हक़
</p>
 : Casual Leave (CL), Privilege Leave (PL), Medical Leave, Leave Without Pay (LWP), Sabbatical, and Compensatory Off (Comp Off).

**********************************************



 Please ask your query related to Haqdarshak's HR Policies. 
 If you are done then type Exit or Quit to finish.

🗣️ : Full and final settlement period


...


<p style="color: hotpink; background-color: purple; padding: 1px; display: inline-block; font-size: 10px; font-weight: bold;">
    हक़
</p>
 : Upon their separation from the company.

**********************************************



 Please ask your query related to Haqdarshak's HR Policies. 
 If you are done then type Exit or Quit to finish.

🗣️ : After how many days FnF will get settled?


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 31.783645134s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 20
}

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 27.62126821s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 20
}
, retry_delay {
  seconds: 27
}
]